<a href="https://colab.research.google.com/github/kimgoinghard/study-machine-learning/blob/main/study_torch_10_softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#One-hot Encoding(원핫인코딩)
-선택해야 하는 개수만큼 차원을 가지면서 정답의 인덱스에 해당하는 원소는 1, 나머지 원소는 0을 갖도록 표현하는 방법이다.

-정수 인코딩과 달리 원-핫 인코딩은 분류 문제 모든 클래스 간의 관계를 균등하게 분배한다.

-이진분류를 위한 로지스틱 회귀와 다르게 다중분류를 위해서는 Softmax함수를 사용한다. 

-소프트맥스 함수는 출력의 총합이 1이 되어, 각 출력값을 확률로 생각할 수 있다.

-또한 비용함수로 크로스엔트로피 오차를 사용한다.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
torch.manual_seed(1)

In [ ]:
z  = torch.FloatTensor([1,2,3])

In [ ]:
y_hat = F.softmax(z, dim =0) #z값에 대한 실제 소프트맥스함수 출력값
print(y_hat)

tensor([0.0900, 0.2447, 0.6652])


In [ ]:
print(y_hat.sum().item()) #출력값의 합은 역시 1이다.

1.0


In [ ]:
z = torch.rand(3, 5, requires_grad = True) #3x5텐서 생성, 갱신 가능

In [ ]:
hypo = F.softmax(z, dim = 1) #dim =1 이므로 각 열의 합, 즉 같은 행의 합이 1이 되어야 한다. 
print(hypo)

tensor([[0.1664, 0.1871, 0.1737, 0.2695, 0.2033],
        [0.2002, 0.1783, 0.2218, 0.1944, 0.2054],
        [0.1809, 0.2380, 0.2318, 0.1084, 0.2409]], grad_fn=<SoftmaxBackward0>)


In [ ]:
y = torch.randint(5,(3,)).long() #5이하의 랜덤 숫자로 (3,) 텐서 생성
y

tensor([0, 1, 2])

In [ ]:
y_one_hot = torch.zeros_like(hypo) #hypo, 즉 실제 소프트맥스 함숫값과 형태가 같고 모든 원소는 0인 텐서 생성
y_one_hot.scatter_(1, y.unsqueeze(1),1) # y.unsqueeze(1)을 하게되면 y가 (3, ) 이었으므로 (3,1)이 된다. 
#scatter의 첫인자는 dim = 1, 즉 열에 대해서 수행하라고 알려준 것이고,  세번째 인자에 1을 넣어주어 두번쨰인자의 인덱스에 1을 넣으라고 알려준것이다.
# scatter(차원, 인덱스, 넣을 숫자)

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [ ]:
print(y.unsqueeze(1))

tensor([[0],
        [1],
        [2]])


In [ ]:
print(y_one_hot)

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])


이제 소프트맥스 함수의 비용함수를 만들겠다.

In [ ]:
cost = (y_one_hot*-torch.log(hypo)).sum(dim=1).mean() #sum의 dim이 1, 같은 행의 데이터가 한 입력으로 온것이기 떄문에 열기준 으로 계산해야 행의 합을 계산하는 것임.
print(cost)

tensor(1.6599, grad_fn=<MeanBackward0>)


In [ ]:
torch.log(F.softmax(z, dim=1)) 

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]], grad_fn=<LogBackward0>)

In [ ]:
F.log_softmax(z, dim=1) #위의 함수와 같다. 파이토치에서는 softmax와 log를 결합한 log_softmax함수를 제공한다. 

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]],
       grad_fn=<LogSoftmaxBackward0>)

In [ ]:
(y_one_hot*-torch.log(hypo)).sum(dim=1).mean() #torch.log(hypo)를 log_softmax로 대체할 수 있다. (hypo는 softmax를 통과시킨 값임.)

tensor(1.6599, grad_fn=<MeanBackward0>)


In [ ]:
(y_one_hot*F.log_softmax(z)).sum(dim=1).mean()

<ipython-input-58-1e3ce4f80220>:1: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  (y_one_hot*F.log_softmax(z)).sum(dim=1).mean()


tensor(-1.6599, grad_fn=<MeanBackward0>)

In [ ]:
#more high level
F.nll_loss(F.log_softmax(z,dim=1),y) 

tensor(1.6599, grad_fn=<NllLossBackward0>)

위에서 nll은 negative log likelihood의 약자이다. nll_loss는 F.log_softmax()를 수행한 후에 남은 계산(더하고 평균내는 것)을 수행한다. 

In [ ]:
F.cross_entropy(z,y) # y : one-ht-vector, z : 입력데이터,torchTensor  //// F.cross_entropy는 비용함수에 소프트맥스까지 포함하고 있다.

tensor(1.6599, grad_fn=<NllLossBackward0>)